In [5]:
from keras import backend as K
import time
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import win32com.client as wincl

PADDING = 50
ready_to_detect_identity = True

FRmodel = faceRecoModel(input_shape=(3, 96, 96))

def triplet_loss(y_true, y_pred, alpha = 0.3):
   
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

def prepare_database():
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob("images/*"):
        identity = os.path.splitext(os.path.basename(file))[0]
        database[identity] = img_path_to_encoding(file, FRmodel)

    return database

def webcam_face_recognizer(database):

    global ready_to_detect_identity

    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(0)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame

        # We do not want to detect a new identity while the program is in the process of identifying another person
        if ready_to_detect_identity:
            img = process_frame(img, frame, face_cascade)   
        
        key = cv2.waitKey(100)
        cv2.imshow("preview", img)

        if key == 27: # exit on ESC
            break
    cv2.destroyWindow("preview")

def process_frame(img, frame, face_cascade):
    """
    Determine whether the current frame contains the faces of people from our database
    """
    global ready_to_detect_identity
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop through all the faces detected and determine whether or not they are in the database
    identities = []
    for (x, y, w, h) in faces:
        x1 = x-PADDING
        y1 = y-PADDING
        x2 = x+w+PADDING
        y2 = y+h+PADDING

        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),2)
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(img,'name',(0,130), font, 1, (200,255,155)) #---write the text

        identity = find_identity(frame, x1, y1, x2, y2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,identity,(0,130), font, 1, (200,255,155))

        if identity is not None:
            identities.append(identity)

    if identities != []:
        cv2.imwrite('example.png',img)
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(img,'name',(0,130), font, 1, (200,255,155)) #---write the text
        #cv2.imshow('Face having name', img)

        ready_to_detect_identity = False
        #pool = Pool(processes=1) 
        # We run this as a separate process so that the camera feedback does not freeze
        #pool.apply_async(welcome_users, [identities])
    return img

def find_identity(frame, x1, y1, x2, y2):
   
    height, width, channels = frame.shape
    # The padding is necessary since the OpenCV face detector creates the bounding box around the face and not the head
    part_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
    
    return who_is_it(part_image, database, FRmodel)

def who_is_it(image, database, model):
    
   
    encoding = img_to_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dist = np.linalg.norm(db_enc - encoding)

        print('distance for %s is %s' %(name, dist))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.5:
        return None
    else:
        return str(identity)



    ready_to_detect_identity = True

if __name__ == "__main__":
    database = prepare_database()
    webcam_face_recognizer(database)


distance for andrew is 0.8780976
distance for camera_0 is 1.0376691
distance for danielle is 1.1282339
distance for felix is 0.7202981
distance for Gagandeep is 0.8481864
distance for kevin is 0.74335074
distance for skuli is 0.85644686
distance for tian is 1.0928156
distance for younes is 1.2048938
distance for andrew is 0.80829126
distance for camera_0 is 1.0360473
distance for danielle is 0.8851891
distance for felix is 0.83464277
distance for Gagandeep is 0.9683932
distance for kevin is 0.8148407
distance for skuli is 0.89996356
distance for tian is 0.8442188
distance for younes is 1.1676627
distance for andrew is 0.874565
distance for camera_0 is 1.0447826
distance for danielle is 0.9298893
distance for felix is 0.8408871
distance for Gagandeep is 1.0137533
distance for kevin is 0.78566366
distance for skuli is 0.9159146
distance for tian is 0.8612434
distance for younes is 1.162992
distance for andrew is 0.81716776
distance for camera_0 is 1.0371693
distance for danielle is 0.830

In [1]:
from numpy import *
import matplotlib as plt
import cv2

img = cv2.imread('images/andrew.jpg')
print (img.shape)

cv2.imshow('Amanda', img)

(96, 96, 3)


In [4]:
loss

NameError: name 'loss' is not defined